# Create a spreadsheet with the results

En este notebook se genera una hoja de calculo con los datos generados por los notebooks 


In [1]:
from configparser import ConfigParser, ExtendedInterpolation
import os
from functools import reduce

import numpy as np
import pandas as pd
import h5py

from spikelib.utils import datasets_to_array

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

['../../config.ini']

In [2]:
experiment = config['EXP']['name']
processed_file = config['FILES']['processed']
sorting_file = config['FILES']['sorting']
outút_folder = config['REPORT']['spreadsheet']
events_file = config['SYNC']['events']

In [3]:
print('To export {} experiment to csv file'.format(experiment))   

To export MR-0061 experiment to csv file


## Scotopic and photopic

In [4]:
df = pd.read_csv(events_file)
protocol_fr = []
protocol_names = ['scotopic', 'photopic']

for idx, event in df[df['protocol_name'].isin(protocol_names)].iterrows():
    start_protocol = event['start_event']
    end_protocol = event['end_event']
    
    protocol_rate = {}
    with h5py.File(sorting_file) as fsorting: 
        for key, spikes in fsorting['spiketimes'].items():
            spk = spikes[...]
            filter_spikes = np.logical_and(spk > start_protocol, spk < end_protocol)
            protocol_rate[key] = spk[filter_spikes].size/((end_protocol - start_protocol)/20000.)

    protocol_fr.append(pd.Series(protocol_rate, name='fr_{}_{}_{}'.format(event['protocol_name'], event['nd'], event['extra_desciption'])))
protocol_frs = reduce(lambda x, y: pd.concat([x, y], axis=1), protocol_fr)

In [5]:
protocol_frs.sample(5)

,fr_scotopic_nd3_initial,fr_scotopic_nd3_final
temp_185,0.903136,2.618299
temp_39,0.353976,5.983042
temp_264,0.211724,1.297666
temp_48,0.306008,6.809873
temp_58,0.076088,1.550308


## Temporal properties from the Checkerboard stimulus

In [6]:
# Source file
intensity_sta = 'nd3-255'
temp_char_group = '/sta/{}/temporal/char/'.format(intensity_sta)
valid_group = '/sta/{}/valid/'.format(intensity_sta)
with h5py.File(processed_file, 'r') as panalysis:
    col_name = panalysis[temp_char_group].attrs['col_name'].split(',')
    
    data_char = {}
    valid = {}
    for key in panalysis[temp_char_group]:
        data_char[key] = panalysis[temp_char_group+key][...]
        valid[key] = panalysis[valid_group+key][...]
    
    wn_temp_char = pd.DataFrame(data_char, index=col_name).transpose()
    wn_temp_char['validsta'] = pd.Series(valid)


wn_temp_char['statype'].replace({1:'ON',0:'OFF'},inplace=True)

# Partial result
wn_col_name = {'peakamplitud': 'RF_PeakFrame_Amp',
               'peaktime': 'RF_PeakFrame',
               'peaktime2': 'RF_PeakFrame2',
               'biphasicindex': 'RF_BiphasicIndex',
               'zerocrossing': 'RF_ZeroCrossing',
               'statype': 'RF_STAtype',
               'validsta': 'RF_valid',
              }

wn_temp_char.rename(columns=wn_col_name,inplace=True)
# results = pd.concat([results, wn_temp_char],axis=1)

In [7]:
wn_temp_char.sample(5)

,RF_PeakFrame,RF_PeakFrame2,RF_PeakFrame_Amp,RF_BiphasicIndex,RF_ZeroCrossing,RF_STAtype,RF_valid
temp_63,-0.075034,-0.176518,-0.31695,0.292217,-0.125776,OFF,True
temp_311,NaN,NaN,NaN,NaN,NaN,NaN,False
temp_168,NaN,NaN,NaN,NaN,NaN,NaN,False
temp_98,NaN,NaN,NaN,NaN,NaN,NaN,False
temp_295,NaN,NaN,NaN,NaN,NaN,NaN,False


## Firering rate fo Checkerboard

In [8]:
df = pd.read_csv(events_file)
wn_fr = []
protocol_name = [config['CHECKERBOARD']['protocol_name']]

for idx, event in df[df['protocol_name'].isin(protocol_name)].iterrows():
    start_wn = event['start_event']
    end_wn = event['end_event']
    
    wn_rate = {}
    with h5py.File(sorting_file) as fprocessed: 
        for key, spikes in fprocessed['spiketimes'].items():
            spk = spikes[...]
            filter_spikes = np.logical_and(spk > start_wn, spk < end_wn)
            wn_rate[key] = spk[filter_spikes].size/((end_wn - start_wn)/20000.)

    wn_fr.append(pd.Series(wn_rate, name='fr_{}_{}_{}'.format(event['protocol_name'], event['nd'], event['extra_desciption'])))
wn_frs = reduce(lambda x, y: pd.concat([x, y], axis=1), wn_fr)

In [9]:
wn_frs

temp_0       2.121244
temp_1       0.787369
temp_10      2.083038
temp_100     9.744656
temp_101     3.957330
              ...    
temp_95      3.480590
temp_96      3.765748
temp_97     11.644971
temp_98      6.342138
temp_99      1.147277
Name: fr_checkerboard_nd3_cyan, Length: 350, dtype: float64

## Temporal fit for WN

In [10]:
# Source file
temp_fit_pars_group = '/sta/{}/temporal/fit_params/'.format(intensity_sta)

with h5py.File(processed_file, 'r') as panalysis:
    col_name = panalysis[temp_fit_pars_group].attrs['col_name'].split(',')
    
    fit_pars = {}
    for key in panalysis[temp_fit_pars_group]:
        fit_pars[key] = panalysis[temp_fit_pars_group+key][...]
    
    wn_temp_fit = pd.DataFrame(fit_pars, index=col_name).transpose()
    rename = ['RF_Fit_amp1', 'RF_Fit_amp2', 'RF_Fit_tau1', 'RF_Fit_tau2', 'RF_Fit_n']
    rename = {col_name[k]: rename[k] for k in range(len(col_name))}
    wn_temp_fit.rename(columns=rename,inplace=True)

wn_temp_fit.sample(5)
# results = pd.concat([results, wn_temp_fit],axis=1)

,RF_Fit_amp1,RF_Fit_amp2,RF_Fit_tau1,RF_Fit_tau2,RF_Fit_n
temp_92,-0.010317,0.172124,-0.468555,-0.068662,12.951551
temp_26,-0.000000,0.000000,-0.000000,-0.000000,0.000000
temp_221,0.000000,-0.000000,-0.000000,-0.000000,0.000000
temp_268,0.000000,0.000000,-0.000000,-0.000000,0.000000
temp_161,0.000000,0.000000,-0.000000,-0.000000,0.000000


## Spatial properties from the White Noise stimulus

In [11]:
block_to_um = 50  # 12 pixel x 1 block x  4.1666 micrometers
spatial_char_group = '/sta/{}/spatial/char/'.format(intensity_sta)

with h5py.File(processed_file, 'r') as panalysis:
    col_name = panalysis[spatial_char_group].attrs['col_name'].split(',')
    array, key = datasets_to_array(panalysis[spatial_char_group])
#     spatial_char = {key: panalysis[spatial_char_group+key][...] for key in panalysis[spatial_char_group]}

wn_spatial = pd.DataFrame(data=array, index=key, columns=col_name)


rename_columns = {
    'angle': 'RF_rotation',
    'a': 'RF_semia',
    'b': 'RF_semib',
    'x': 'RF_x',
    'y': 'RF_y',
    'snr': 'RF_snr',
    'frame': 'RF_frame_maxr'
}
wn_spatial.rename(columns=rename_columns, inplace=True)
wn_spatial['RF_semia'].iloc[:] = wn_spatial['RF_semia']*block_to_um
wn_spatial['RF_semib'].iloc[:] = wn_spatial['RF_semib']*block_to_um
wn_spatial['RF_x'].iloc[:] = wn_spatial['RF_x']*block_to_um
wn_spatial['RF_y'].iloc[:] = wn_spatial['RF_y']*block_to_um
wn_spatial['RF_area'] = wn_spatial['RF_semia'] * wn_spatial['RF_semib'] * np.pi
wn_spatial['RF_radmax'] = wn_spatial[['RF_semia','RF_semib']].max(axis=1)


## Linearity for the white noise stimulus

## Temporal propierties of the Flash stimulus

In [12]:
flash_result = []
with h5py.File(processed_file, 'r') as panalysis:
    for kfilter in panalysis['/flash/']:
        for intensity in panalysis['/flash/' + kfilter]:
            flash_char_group = '/flash/{}/{}/char/'.format(kfilter, intensity)
            flash_type_group = '/flash/{}/{}/type/'.format(kfilter, intensity)
            col_name = panalysis[flash_char_group].attrs['col_name'].split(',')
            col_name = ['FL_{}_{}_{}'.format(kfilter, intensity, k) for k in col_name]
            array, key = datasets_to_array(panalysis[flash_char_group])
            flash_char = pd.DataFrame(data=array, index=key, columns=col_name, )

            type_, key_type = datasets_to_array(panalysis[flash_type_group])
            flash_char['FL_{}_{}_type'.format(kfilter, intensity)] = pd.Series(type_, index=key_type)
            flash_char['FL_{}_{}_type'.format(kfilter, intensity)] = \
                flash_char['FL_{}_{}_type'.format(kfilter, intensity)].map(
                {np.nan: 'Null', 0: 'Null', 1: 'On', 2: 'Off', 3: 'OnOff'},
            )
            flash_result.append(flash_char)
flash_results = reduce(lambda x, y: pd.concat([x, y], axis=1), flash_result)

In [13]:
flash_results.sample(5)

,FL_nd3_050_cyan_latency_on,FL_nd3_050_cyan_latency_off,FL_nd3_050_cyan_bias_idx,FL_nd3_050_cyan_decay_on,FL_nd3_050_cyan_decay_off,FL_nd3_050_cyan_resp_index_on,FL_nd3_050_cyan_resp_index_off,FL_nd3_050_cyan_sust_index_on,FL_nd3_050_cyan_sust_index_off,FL_nd3_050_cyan_frmax_on,...,FL_nd3_255_cyan_bias_idx,FL_nd3_255_cyan_decay_on,FL_nd3_255_cyan_decay_off,FL_nd3_255_cyan_resp_index_on,FL_nd3_255_cyan_resp_index_off,FL_nd3_255_cyan_sust_index_on,FL_nd3_255_cyan_sust_index_off,FL_nd3_255_cyan_frmax_on,FL_nd3_255_cyan_frmax_off,FL_nd3_255_cyan_type
temp_198,0.139113,0.0,1.0,0.059048,0.0,0.916954,0.0,0.398111,0.0,0.419355,...,1.0,0.011009,0.0,0.896567,0.0,0.603675,0.0,0.838710,0.0,On
temp_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Null
temp_165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Null
temp_173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.012010,0.0,0.883615,0.0,0.582312,0.0,0.741935,0.0,On
temp_347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Null


## Save results to CSV file

In [14]:
file_name = '{}{}.csv'.format(outút_folder, experiment)

results = reduce(lambda x, y: pd.concat([x, y],axis=1),
                 [protocol_frs, wn_temp_char, wn_frs, wn_temp_fit, wn_spatial, flash_results])

results.to_csv(file_name, float_format='%.10f')
print('Saved in {}'.format(file_name))

results.head()

Saved in /home/vnc/Downloads/MR-0061/reports/spreadsheets/MR-0061.csv


,fr_scotopic_nd3_initial,fr_scotopic_nd3_final,RF_PeakFrame,RF_PeakFrame2,RF_PeakFrame_Amp,RF_BiphasicIndex,RF_ZeroCrossing,RF_STAtype,RF_valid,fr_checkerboard_nd3_cyan,...,FL_nd3_255_cyan_bias_idx,FL_nd3_255_cyan_decay_on,FL_nd3_255_cyan_decay_off,FL_nd3_255_cyan_resp_index_on,FL_nd3_255_cyan_resp_index_off,FL_nd3_255_cyan_sust_index_on,FL_nd3_255_cyan_sust_index_off,FL_nd3_255_cyan_frmax_on,FL_nd3_255_cyan_frmax_off,FL_nd3_255_cyan_type
temp_0,0.074434,1.584760,-0.485289,-0.485289,NaN,NaN,-0.485289,OFF,True,2.121244,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Null
temp_1,0.347360,0.734961,NaN,NaN,NaN,NaN,NaN,NaN,False,0.787369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Null
temp_10,0.388712,2.974295,NaN,NaN,NaN,NaN,NaN,NaN,False,2.083038,...,-1.0,0.000000,0.011009,0.000000,0.799465,0.000000,0.89988,0.000000,1.16129,Off
temp_100,5.789332,2.193399,NaN,NaN,NaN,NaN,NaN,NaN,False,9.744656,...,1.0,0.015012,0.000000,0.745243,0.000000,0.675598,0.00000,0.806452,0.00000,On
temp_101,2.051078,6.878776,NaN,NaN,NaN,NaN,NaN,NaN,False,3.957330,...,1.0,0.009007,0.000000,0.951149,0.000000,0.756772,0.00000,0.774194,0.00000,On
